# Useful `combinator` functions

We can apply clone to model instances that have child layers, making it easy to define more complex architectures. 
For instance, we often want to attach an activation function and dropout to a linear layer, and then repeat that 
substructure a number of times. Of course, you can make whatever intermediate functions you find helpful.


In [ ]:
!pip install "thinc>=8.0.0a0"

In [ ]:
import numpy
from thinc.api import (Linear, 
                       chain, 
                       concatenate, 
                       clone, 
                       glorot_uniform_init, 
                       zero_init,
                       Relu, 
                       Dropout,
                       with_array)


The `chain` function wires two model instances together, with a feed-forward relationship.

In [ ]:
n_hidden = 128
X = numpy.zeros((128, 16), dtype="f")
Y = numpy.zeros((128, 10), dtype="f")

model = chain(Linear(n_hidden, init_W=glorot_uniform_init), Linear(init_W=zero_init),)
model.initialize(X=X, Y=Y)
nI = model.get_dim("nI")
nO = model.get_dim("nO")
nO_hidden = model.layers[0].get_dim("nO")
print(f"Initialized model with input dimension nI={nI} and output dimension nO={nO}.")
print(f"The size of the hidden layer is {nO_hidden}.")

The `concatenate` combinator produces a layer that runs the child layers separately, and then concatenates
their outputs together. This is often useful for combining features from different sources.  

In [ ]:

model = concatenate(Linear(n_hidden), Linear(n_hidden))
model.initialize(X=X)

ni_layer_0 = model.layers[0].get_dim("nI")
no_layer_0 = model.layers[0].get_dim("nO")
print(f"The input size of the hidden layer 0 is {ni_layer_0}.")
print(f"The output size of the hidden layer 0 is {no_layer_0}.")

ni_layer_1 = model.layers[1].get_dim("nI")
no_layer_1 = model.layers[1].get_dim("nO")
print(f"The input size of the hidden layer 1 is {ni_layer_1}.")
print(f"The output size of the hidden layer 1 is {no_layer_1}.")

nI = model.get_dim("nI")
nO = model.get_dim("nO")
nO_hidden = model.layers[0].get_dim("nO")

print(f"Initialized model with input dimension nI={nI}.")
print(f"Initialized model with output dimension nO={nO}.")

The `clone` combinator creates a number of copies of a layer, and chains them together into a 
deep feed-forward network.

The shape inference is especially handy here: we want the first and last layers to have different shapes, 
so we can avoid providing any dimensions into the layer we clone. We then just have to specify the 
first layer's output size, and we can let the rest of the dimensions be inferred from the data.

In [ ]:
model = clone(Linear(), 5)
model.layers[0].set_dim("nO", n_hidden)
model.initialize(X=X, Y=Y)
nI = model.get_dim("nI")
nO = model.get_dim("nO")
print(f"Initialized model with input dimension nI={nI} and output dimension nO={nO}.")

We can apply clone to model instances that have child layers, making it easy to define more 
complex architectures. For instance, we often want to attach an *activation function* and *dropout* to 
a linear layer, and then repeat that substructure a number of times. 



In [ ]:
def MyCustomLayer(dropout=0.2):
    return chain(Linear(), Relu(), Dropout(dropout))

model = clone(MyCustomLayer(0.2), 5)

Thinc also provides several *input and output transformation* combinators as unary functions.

The `with_array` combinator produces a model that *flattens lists of arrays* into a single array, 
and then *calls the child layer* to get the flattened output. It then *reverses* the transformation on 
the output.

 

In [ ]:
nI=4
nO=2
n_instances=10

model = with_array(Linear(nO, nI))

# allocate a 10 instances of 2-dimensional arrays of float elements
Xs = [model.ops.alloc2f(n_instances, nI, dtype="f")]

model.initialize(X=Xs)
Ys = model.predict(Xs)
print(f"Prediction shape: {Ys[0].shape}.")

What it does is effectively compute `numpy.hstack()` on the input arrays before passing in to the child layer.
However, it worth mentioning the first array's dimension dictates the size of the transformation.
Thus, the feature dimension should match. 
For example:

In [ ]:
model = with_array(Linear(nO, nI))

# allocate 2
Xs = [model.ops.alloc2f(6, nI, dtype="f"), model.ops.alloc2f(9, nI, dtype="f")]

print("Xs:")
print(Xs)

print(f"first array shape '{Xs[0].shape}' and type '{type(Xs[0])}'")
print("The first array:")
print(Xs[0])
print("---")
print(f"second array shape '{Xs[1].shape}' and type '{type(Xs[1])}'")
print("The second array:")
print(Xs[1])

model.initialize(X=Xs)
Ys = model.predict(Xs)
print(f"Prediction shape will be: {Ys[0].shape}.")